In [1]:
import torch 
from torch import nn,optim
import torchvision 
from torchvision import datasets,transforms
from torch.utils.data import DataLoader 
from torch.utils.tensorboard import SummaryWriter 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
### Discriminator has no fc layers
class Discriminator(nn.Module):

    def __init__(self, channels_img, features_d):
        super(Discriminator,self).__init__()

        # Input shape : N x channels_img x 64 x 64
        self.disc = nn.Sequential(
            nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1), # 32x32
            nn.LeakyReLU(0.2),

            self._block(features_d, features_d*2, 4, 2, 1), # 16x16
            self._block(features_d*2, features_d*4, 4, 2, 1), # 8x8
            self._block(features_d*4, features_d*8, 4, 2, 1), # 4x4

            nn.Conv2d(features_d*8, 1, kernel_size=4, stride=2, padding=0) # 1x1
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):

        return nn.Sequential(
            nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding,bias=False),
            nn.InstanceNorm2d(out_channels, affine=True),
            nn.LeakyReLU(0.2)
        )

    def forward(self,x):
        return self.disc(x)

In [4]:
class Generator(nn.Module):

    def __init__(self, z_dim, channels_img, features_g):
        super(Generator,self).__init__()

        self.gen = nn.Sequential(

            self._block(z_dim, features_g*16, 4, 1, 0), # 4x4
            self._block(features_g*16, features_g*8, 4, 2, 1), # 8x8
            self._block(features_g*8, features_g*4, 4, 2, 1), # 16x16
            self._block(features_g*4, features_g*2, 4, 2, 1), # 32x32

            nn.ConvTranspose2d(features_g*2, channels_img, kernel_size=4, stride=2, padding=1), # 64x64
            nn.Tanh()

        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):

        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self,x):
        return self.gen(x)

In [28]:
def gradient_penalty(critic, real, fake, device="cpu"):
    BATCH_SIZE, C, H, W = real.shape
    alpha = torch.rand((BATCH_SIZE, 1, 1, 1)).repeat(1, C, H, W).to(device)
    interpolated_images = real * alpha + fake * (1 - alpha)

    # Calculate critic scores
    mixed_scores = critic(interpolated_images)

    # Take the gradient of the scores with respect to the images
    gradient = torch.autograd.grad(
        inputs=interpolated_images,
        outputs=mixed_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True,
    )[0]
    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim=1)
    gradient_penalty = torch.mean((gradient_norm - 1) ** 2)
    return gradient_penalty

In [6]:
def initialize_weights(model):
    for m in model.modules() :
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)

In [7]:
def test():
    N, in_channels, H, W = 8, 3, 64, 64
    z_dim = 100 
    x = torch.randn((N, in_channels, H, W))

    disc = Discriminator(in_channels, 8)
    initialize_weights(disc)
    assert disc(x).shape == (N, 1, 1, 1)

    gen = Generator(z_dim, in_channels, 8)
    initialize_weights(gen)

    z = torch.randn((N, z_dim, 1, 1))
    assert gen(z).shape == (N, in_channels, H, W)

In [8]:
BATCH_SIZE = 64
LR = 1e-4
CHANNELS_IMG = 3
FEATURES_D = 64
Z_DIM = 100
FEATURES_G = 64
IMAGE_SIZE = 64
step = 0
CRITIC_ITERATIONS = 5
LAMBDA_GP = 10

In [9]:
dev = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

t = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)])
])

#data = datasets.MNIST('./data', transform=t, download=True, train=True)
data = datasets.ImageFolder('/content/drive/MyDrive/dataset', transform=t)
loader = DataLoader(dataset=data, batch_size=BATCH_SIZE, shuffle=True)
print(data)

Dataset ImageFolder
    Number of datapoints: 5909
    Root location: /content/drive/MyDrive/dataset
    StandardTransform
Transform: Compose(
               Resize(size=64, interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )


In [10]:
disc = Discriminator(CHANNELS_IMG, FEATURES_D)
#disc.load_state_dict(torch.load('/content/drive/MyDrive/WGAN/disc123.pt', map_location=dev))
disc.to(dev)
initialize_weights(disc)
opt_disc = optim.Adam(disc.parameters(), lr = LR, betas = (0.0,0.9))

In [11]:
gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_G)
#gen.load_state_dict(torch.load('/content/drive/MyDrive/WGAN/gen123.pt', map_location=dev))
gen.to(dev)
initialize_weights(gen)
opt_gen = optim.Adam(gen.parameters(), lr = LR, betas = (0.0,0.9))

In [12]:
fixed_noise = torch.randn((32, Z_DIM, 1, 1)).to(dev)

writer_real = SummaryWriter(log_dir='runs/real')
writer_fake = SummaryWriter(log_dir='runs/fake')

gen.train()
disc.train()

Discriminator(
  (disc): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (4): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (5): Conv2d(512, 1, kernel_size=(4, 4), stride=(2, 2))
  )
)

In [ ]:
EPOCHS = 200
for epoch in range(EPOCHS):
    for batch_idx, (real, _) in enumerate(loader):
        if batch_idx == 92 : 
          break
        real = real.to(dev)

        for _ in range(CRITIC_ITERATIONS):
            noise = torch.randn((BATCH_SIZE, Z_DIM, 1, 1)).to(dev)
            fake = gen(noise)

            disc_real = disc(real).reshape(-1)
            disc_fake = disc(fake).reshape(-1)

            gp = gradient_penalty(disc, real, fake, device = dev)

            loss_D = (-(torch.mean(disc_real) - torch.mean(disc_fake)) + LAMBDA_GP * gp)

            disc.zero_grad()
            loss_D.backward(retain_graph=True)
            opt_disc.step()

        # train gen
        output = disc(fake).reshape(-1)
        loss_G = -torch.mean(output)
        gen.zero_grad()
        loss_G.backward()
        opt_gen.step()

        if batch_idx % 92 == 0:
            print(
                f"Epoch [{epoch}/{EPOCHS}] Batch {batch_idx}/{len(loader)} \
                  Loss D: {loss_D:.4f}, loss G: {loss_G:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise)
                # take out (up to) 32 examples
                img_grid_real = torchvision.utils.make_grid(
                    real[:32], normalize=True
                )
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:32], normalize=True
                )

                writer_real.add_image("Real", img_grid_real, global_step=step)
                writer_fake.add_image("Fake", img_grid_fake, global_step=step)

            step += 1

    torch.save(disc.state_dict(), f'/content/drive/MyDrive/WGAN_GP/disc{step}.pt')
    torch.save(gen.state_dict(), f'/content/drive/MyDrive/WGAN_GP/gen{step}.pt')


Epoch [0/200] Batch 0/93                   Loss D: -16.4660, loss G: 97.9360
Epoch [1/200] Batch 0/93                   Loss D: -15.9266, loss G: 92.6589
Epoch [2/200] Batch 0/93                   Loss D: -16.6279, loss G: 87.3288
Epoch [3/200] Batch 0/93                   Loss D: -16.9854, loss G: 92.9295
Epoch [4/200] Batch 0/93                   Loss D: -17.5651, loss G: 90.6648
Epoch [5/200] Batch 0/93                   Loss D: -13.7457, loss G: 89.0549
Epoch [6/200] Batch 0/93                   Loss D: -13.1712, loss G: 87.7857
Epoch [7/200] Batch 0/93                   Loss D: -12.5203, loss G: 91.0849
Epoch [8/200] Batch 0/93                   Loss D: -14.5989, loss G: 84.5986
Epoch [9/200] Batch 0/93                   Loss D: -13.7905, loss G: 88.8603
Epoch [10/200] Batch 0/93                   Loss D: -15.3539, loss G: 84.3146
Epoch [11/200] Batch 0/93                   Loss D: -11.8278, loss G: 95.0411
Epoch [12/200] Batch 0/93                   Loss D: -13.5490, loss G: 84.8